## 3 Generating centroid profiles
In this notebook, we identify the centroid for each cluster found in the single-cell profiles after running the buscar clustering module.

The centroid is the representative cell that best captures the distribution of cells within a cluster.

In [1]:
import sys
import pathlib

import polars as pl
from pycytominer import aggregate

sys.path.append("../../")
from utils.data_utils import split_meta_and_features

Setting input and output paths

In [2]:
# setting data path for cfret-pilot dataset
cfret_profiles_path = pathlib.Path(
    "../0.download-data/data/sc-profiles/cfret/localhost230405150001_sc_feature_selected.parquet"
).resolve(strict=True)

# setting cluster labels path
cluster_labels_path = pathlib.Path(
    "./results/clusters/cfret_pilot_cluster_labels.parquet"
).resolve(strict=True)

# setting outpaths for results
results_dir = pathlib.Path("./results").resolve()
results_dir.mkdir(exist_ok=True)

# setting outpath for centroids
centroids_dir = (results_dir / "centroids").resolve()
centroids_dir.mkdir(exist_ok=True)

Loading profiles

In [3]:
# loading in profiles and add cluster labels to profiles dataframe
cfret_df = pl.read_parquet(cfret_profiles_path).join(
    pl.read_parquet(cluster_labels_path), on="Metadata_cell_id", how="inner"
)

# add a column that indicates the heart and treatment added
cfret_df = cfret_df.with_columns(
    pl.concat_str(
        [
            pl.col("Metadata_cell_type"),
            pl.col("Metadata_treatment"),
        ],
        separator="_",
    ).alias("Metadata_heart_treatment")
)

# split feature space
cfret_meta, cfret_feats = split_meta_and_features(cfret_df)

# display
print(cfret_df.shape)


cfret_df.select(cfret_meta + cfret_feats).head()

(15793, 682)


Metadata_cell_id,Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_cell_type,Metadata_heart_failure_type,Metadata_treatment,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,Metadata_Image_Count_Cells,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cells_Number_Object_Number,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_Nuclei_Number_Object_Number,Metadata_Site,Metadata_cluster_id,Metadata_cluster_n_cells,Metadata_treatment_n_cells,Metadata_cluster_ratio,Metadata_heart_treatment,Cytoplasm_AreaShape_BoundingBoxMinimum_X,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_Eccentricity,Cytoplasm_AreaShape_Extent,Cytoplasm_AreaShape_FormFactor,Cytoplasm_AreaShape_MajorAxisLength,Cytoplasm_AreaShape_MeanRadius,Cytoplasm_AreaShape_MinorAxisLength,Cytoplasm_AreaShape_Perimeter,Cytoplasm_AreaShape_Solidity,Cytoplasm_AreaShape_Zernike_0_0,Cytoplasm_AreaShape_Zernike_1_1,…,Nuclei_Texture_DifferenceEntropy_PM_3_00_256,Nuclei_Texture_DifferenceVariance_Actin_3_01_256,Nuclei_Texture_DifferenceVariance_Mitochondria_3_03_256,Nuclei_Texture_DifferenceVariance_PM_3_03_256,Nuclei_Texture_InfoMeas1_ER_3_00_256,Nuclei_Texture_InfoMeas1_ER_3_01_256,Nuclei_Texture_InfoMeas1_ER_3_02_256,Nuclei_Texture_InfoMeas1_ER_3_03_256,Nuclei_Texture_InfoMeas1_Hoechst_3_00_256,Nuclei_Texture_InfoMeas1_Hoechst_3_01_256,Nuclei_Texture_InfoMeas1_Hoechst_3_02_256,Nuclei_Texture_InfoMeas1_Hoechst_3_03_256,Nuclei_Texture_InfoMeas1_Mitochondria_3_00_256,Nuclei_Texture_InfoMeas1_Mitochondria_3_01_256,Nuclei_Texture_InfoMeas1_Mitochondria_3_02_256,Nuclei_Texture_InfoMeas1_Mitochondria_3_03_256,Nuclei_Texture_InfoMeas1_PM_3_00_256,Nuclei_Texture_InfoMeas1_PM_3_01_256,Nuclei_Texture_InfoMeas1_PM_3_02_256,Nuclei_Texture_InfoMeas1_PM_3_03_256,Nuclei_Texture_InfoMeas2_ER_3_01_256,Nuclei_Texture_InfoMeas2_ER_3_03_256,Nuclei_Texture_InfoMeas2_Hoechst_3_01_256,Nuclei_Texture_InfoMeas2_Hoechst_3_03_256,Nuclei_Texture_InfoMeas2_PM_3_01_256,Nuclei_Texture_InfoMeas2_PM_3_03_256,Nuclei_Texture_InverseDifferenceMoment_Actin_3_02_256,Nuclei_Texture_InverseDifferenceMoment_ER_3_01_256,Nuclei_Texture_InverseDifferenceMoment_ER_3_03_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_03_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_01_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_03_256,Nuclei_Texture_SumEntropy_PM_3_01_256,Nuclei_Texture_SumVariance_ER_3_03_256,Nuclei_Texture_SumVariance_Hoechst_3_03_256,Nuclei_Texture_SumVariance_Mitochondria_3_01_256,Nuclei_Texture_SumVariance_PM_3_01_256
str,str,i64,i64,str,str,str,f64,f64,f64,f64,i64,i64,str,str,i64,i64,i64,i64,str,cat,u32,u32,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""210e4376efde9d557a5c60029bdda6…","""B""",2,9,"""failing""","""rejected""","""DMSO""",221.046761,137.115493,246.6028,109.285755,40,1,"""localhost230405150001""","""B02""",1,1,6,6,"""f00""","""DMSO_heart_9_louvain_0""",9142,9153,0.998798,"""failing_DMSO""",-1.35494,0.841229,0.648883,-0.850138,-1.045214,1.298358,0.376165,0.935101,1.530228,-0.983617,-0.261031,-0.299817,…,-0.740763,-0.052719,0.797095,0.359081,-0.173336,0.300041,0.217945,-0.039774,0.488531,0.472164,0.28659,0.464359,0.501649,0.507623,1.076663,0.741941,-0.696022,-0.178762,0.186741,0.158222,0.341595,0.50487,-0.440604,-0.426966,0.194372,-0.035117,0.400021,-0.619206,-0.393448,0.961214,0.406068,0.374039,-0.280532,-0.158967,-0.344804,-0.263653,-0.305486
"""cef18f209640ef8ae98ec110cfdcb6…","""B""",2,9,"""failing""","""rejected""","""DMSO""",690.596142,183.067828,716.170091,177.132195,40,1,"""localhost230405150001""","""B02""",2,2,7,7,"""f00""","""DMSO_heart_9_louvain_0""",9142,9153,0.998798,"""failing_DMSO""",0.657107,-0.850399,-0.584931,2.090925,1.263259,-0.021031,1.627957,0.944161,-0.085511,1.475345,2.1647

We use **median aggregation** to generate centroid profiles for each cluster. For each cluster, we calculate the component-wise median across all cells to create a synthetic representative profile that captures the central tendency. This approach is robust to outliers, consistent with replicate and consensus profile generation workflow, and works well for high-dimensional morphological features.

In [4]:
# split metadata and features
cfret_meta, cfret_feats = split_meta_and_features(cfret_df)

print(f"Total cells: {len(cfret_df)}")
print(f"Number of features: {len(cfret_feats)}")
print(f"Unique clusters: {cfret_df['Metadata_cluster_id'].n_unique()}")

Total cells: 15793
Number of features: 657
Unique clusters: 9


Save centroid profiles

In [5]:
# aggregate by cluster using median to generate centroid profiles
centroids_df = aggregate(
    population_df=cfret_df.to_pandas(),
    strata=["Metadata_cluster_id"],
    features=cfret_feats,
    operation="median",
)

# convert back to polars
centroids_df = pl.from_pandas(centroids_df)

print(f"Total centroids generated: {len(centroids_df)}")
print(f"Centroid shape: {centroids_df.shape}")
centroids_df

/home/erikserrano/Software/miniconda3/envs/buscar/lib/python3.12/site-packages/pycytominer/aggregate.py:108: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  population_df = population_df.groupby(strata, dropna=False)  # type: ignore[assignment]


Total centroids generated: 9
Centroid shape: (9, 658)


Metadata_cluster_id,Cytoplasm_AreaShape_BoundingBoxMinimum_X,Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_Eccentricity,Cytoplasm_AreaShape_Extent,Cytoplasm_AreaShape_FormFactor,Cytoplasm_AreaShape_MajorAxisLength,Cytoplasm_AreaShape_MeanRadius,Cytoplasm_AreaShape_MinorAxisLength,Cytoplasm_AreaShape_Perimeter,Cytoplasm_AreaShape_Solidity,Cytoplasm_AreaShape_Zernike_0_0,Cytoplasm_AreaShape_Zernike_1_1,Cytoplasm_AreaShape_Zernike_2_0,Cytoplasm_AreaShape_Zernike_2_2,Cytoplasm_AreaShape_Zernike_3_1,Cytoplasm_AreaShape_Zernike_4_0,Cytoplasm_AreaShape_Zernike_4_2,Cytoplasm_AreaShape_Zernike_5_1,Cytoplasm_AreaShape_Zernike_5_3,Cytoplasm_AreaShape_Zernike_6_0,Cytoplasm_AreaShape_Zernike_6_2,Cytoplasm_AreaShape_Zernike_6_4,Cytoplasm_AreaShape_Zernike_7_1,Cytoplasm_AreaShape_Zernike_7_3,Cytoplasm_AreaShape_Zernike_7_5,Cytoplasm_AreaShape_Zernike_8_0,Cytoplasm_AreaShape_Zernike_8_2,Cytoplasm_AreaShape_Zernike_8_4,Cytoplasm_AreaShape_Zernike_8_6,Cytoplasm_AreaShape_Zernike_9_1,Cytoplasm_AreaShape_Zernike_9_3,Cytoplasm_AreaShape_Zernike_9_5,Cytoplasm_AreaShape_Zernike_9_7,Cytoplasm_Correlation_Correlation_Actin_Hoechst,Cytoplasm_Correlation_Correlation_Actin_Mitochondria,Cytoplasm_Correlation_Correlation_Actin_PM,…,Nuclei_Texture_DifferenceEntropy_PM_3_00_256,Nuclei_Texture_DifferenceVariance_Actin_3_01_256,Nuclei_Texture_DifferenceVariance_Mitochondria_3_03_256,Nuclei_Texture_DifferenceVariance_PM_3_03_256,Nuclei_Texture_InfoMeas1_ER_3_00_256,Nuclei_Texture_InfoMeas1_ER_3_01_256,Nuclei_Texture_InfoMeas1_ER_3_02_256,Nuclei_Texture_InfoMeas1_ER_3_03_256,Nuclei_Texture_InfoMeas1_Hoechst_3_00_256,Nuclei_Texture_InfoMeas1_Hoechst_3_01_256,Nuclei_Texture_InfoMeas1_Hoechst_3_02_256,Nuclei_Texture_InfoMeas1_Hoechst_3_03_256,Nuclei_Texture_InfoMeas1_Mitochondria_3_00_256,Nuclei_Texture_InfoMeas1_Mitochondria_3_01_256,Nuclei_Texture_InfoMeas1_Mitochondria_3_02_256,Nuclei_Texture_InfoMeas1_Mitochondria_3_03_256,Nuclei_Texture_InfoMeas1_PM_3_00_256,Nuclei_Texture_InfoMeas1_PM_3_01_256,Nuclei_Texture_InfoMeas1_PM_3_02_256,Nuclei_Texture_InfoMeas1_PM_3_03_256,Nuclei_Texture_InfoMeas2_ER_3_01_256,Nuclei_Texture_InfoMeas2_ER_3_03_256,Nuclei_Texture_InfoMeas2_Hoechst_3_01_256,Nuclei_Texture_InfoMeas2_Hoechst_3_03_256,Nuclei_Texture_InfoMeas2_PM_3_01_256,Nuclei_Texture_InfoMeas2_PM_3_03_256,Nuclei_Texture_InverseDifferenceMoment_Actin_3_02_256,Nuclei_Texture_InverseDifferenceMoment_ER_3_01_256,Nuclei_Texture_InverseDifferenceMoment_ER_3_03_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_03_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_01_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_03_256,Nuclei_Texture_SumEntropy_PM_3_01_256,Nuclei_Texture_SumVariance_ER_3_03_256,Nuclei_Texture_SumVariance_Hoechst_3_03_256,Nuclei_Texture_SumVariance_Mitochondria_3_01_256,Nuclei_Texture_SumVariance_PM_3_01_256
cat,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""DMSO_heart_9_louvain_0""",-0.017365,-0.375948,0.028875,0.200863,-0.04977,-0.228398,0.046875,0.063641,-0.251773,0.202299,0.216303,0.032826,0.18301,0.03706,0.0231,-0.16592,-0.007158,0.028768,-0.048361,-0.142402,-0.033751,-0.093717,-0.005448,-0.063685,-0.063426,-0.149138,-0.033556,-0.129697,-0.116366,-0.05216,-0.084436,-0.103767,-0.092716,-0.06537,0.482194,0.064856,…,-0.02033,-0.471746,-0.436971,-0.282468,-0.135781,-0.088347,-0.131908,-0.059839,0.184521,0.321721,0.178624,0.3202,-0.084185,-0.041163,-0.064363,-0.021267,-0.345281,-0.249803,-0.31299,-0.232883,0.493085,0.481681,-0.209778,-0.211974,0.442342,0.437698,-0.508997,-0.369874,-0.374387,-0.143125,0.106185,0.091377,0.125246,-0.14294,-0.321629,-0.179205,-0.172281
"""DMSO_heart_9_louvain_1""",0.395654,2.378332,-0.985671,-2.826533,-1.423156,-1.207654,-2.215587,-0.621809,-1.44509,-3.881097,-2.090087,-1.561

In [6]:
# create a mapping of cluster_id to heart_treatment (unique per cluster)
cluster_treatment_mapping = cfret_df.select(
    ["Metadata_cluster_id", "Metadata_heart_treatment"]
).unique()

# join centroids with the treatment metadata
centroids_df = centroids_df.join(
    cluster_treatment_mapping, on="Metadata_cluster_id", how="left"
)

# save centroids to parquet file
centroids_output_path = centroids_dir / "cfret_pilot_centroids.parquet"
centroids_df.write_parquet(centroids_output_path)

print(f"Centroids saved to: {centroids_output_path}")
print(f"Final centroid shape: {centroids_df.shape}")

Centroids saved to: /home/erikserrano/Projects/buscar/notebooks/2.cfret-analysis/results/centroids/cfret_pilot_centroids.parquet
Final centroid shape: (9, 659)
